In [127]:
p_storage_dir = "../../storage"
p_notebooks_root = None

In [ ]:
if p_notebooks_root is not None:
    import os
    os.chdir(p_notebooks_root)

In [128]:
import ast
import unicodedata
import re

import pandas as pd
from pathlib import Path

IFG_PRODUZ_STORAGE_DIR = Path(f"{p_storage_dir}/ifg_produz")

## Currículos -> Cidades

Notebook focado na ligação de cada currículo com a sua devida cidade que já foi previamente pré-processada.

In [129]:
curriculos = pd.read_csv(IFG_PRODUZ_STORAGE_DIR / "intermediate/curriculos.csv", delimiter=";")

curriculos.sample(n=5)

,codigo,nome_completo,nome_completo_sem_acento,link,nome_citacoes_bibliograficas,uf_nascimento,cidade_nascimento,resumo,palavra_chave,campus_atual,categoria,aceitando_email,siape,email,data_nascimento,sigla_campus_atual
1701,502,Weder David de Freitas,Weder David de Freitas,http://lattes.cnpq.br/2668844019246293,"FREITAS, Weder D.;DAVID DE FREITAS, WEDER",GO,Bela Vista de Goiás,Doutor em Geografia pela Universidade Federal ...,"[{'palavra': 'geografia', 'correta': 'Geografi...",CÂMPUS GOIÂNIA,Docente,True,2723537,weder.freitas@ifg.edu.br,1983-01-02,GYN
1152,1761,Vanessa Freitas Santos,Vanessa Freitas Santos,http://lattes.cnpq.br/9542828273022611,"SANTOS, V. F.",GO,Itumbiara,Graduada em Licenciatura em Química pelo Insti...,"[{'palavra': 'ensino', 'correta': 'Ensino', 'i...",CÂMPUS ITUMBIARA,TAE,True,2080816,vanessa.santos@ifg.edu.br,1988-08-25,ITU
1292,1649,Neville Julio de Vilasboas e Santos,Neville Julio de Vilasboas e Santos,http://lattes.cnpq.br/8905499707931816,"SANTOS, Neville;SANTOS, NEVILLE JULIO DE VILAS...",SP,Guarulhos,Bacharel (2007) e Licenciado (2008) em Ciência...,"[{'palavra': 'educacao', 'correta': 'Educação'...",CÂMPUS ANÁPOLIS,Docente,True,1849109,neville.santos@ifg.edu.br,1985-12-16,ANA
1198,1555,Alysson Benite de Freitas,Alysson Benite de Freitas,http://lattes.cnpq.br/8468761118527440,"FREITAS, A. B.",SP,Ilha Solteira,Possui graduação em Farmácia pela Universidade...,"[{'palavra': 'quimica', 'correta': 'Química', ...",CÂMPUS ITUMBIARA,Docente,True,1358034,alysson.freitas@ifg.edu.br,1980-11-14,ITU
1538,266,Débora Caldas Marques,Debora Caldas Marques,http://lattes.cnpq.br/1384074705194964,"MARQUES, D. C.",MG,Belo Horizonte,Nutricionista graduada pela Universidade Feder...,"[{'palavra': 'nutricao', 'correta': 'Nutrição'...",CÂMPUS GOIÂNIA OESTE,Docente,True,2201150,debora.marques@ifg.edu.br,1983-07-20,OES


In [130]:
cidades = pd.read_csv(IFG_PRODUZ_STORAGE_DIR / "preprocessed/cidades.csv", delimiter=";", converters={'aliases': ast.literal_eval})

cidades.sample(n=5)

,nome,nome_ascii,sigla_uf,aliases,codigo
92,Carmo do Paranaíba,carmo do paranaiba,MG,"[CARMO DO PARANAIBA, Carmo do Paranaíba]",1cc0c11e-814c-4b3a-88a6-26f77b8782a6
131,Santa Juliana,santa juliana,MG,[Santa Juliana],f6ec4aaf-675a-44fc-99dd-a40ac9cc585c
172,Marília,marilia,SP,[Marília],c3e67bdc-9e71-4c5f-a950-1eedeabd09e3
205,Floriano,floriano,PI,"[FLORIANO, Floriano]",8ac9567e-ad4b-4170-9935-8424cefeaa9c
111,Januária,januaria,MG,[Januária],2f194470-7052-434e-9723-58253a32aeba


### Ligar cada currículo com sua devida cidade de nascimento

In [131]:
def normalizar_string(s):
    if pd.isna(s):
        return
    
    # Transforma todos os caracteres em minúsculas
    s = s.lower()
    
    # Remove acentos das letras
    s = unicodedata.normalize("NFD", s).encode("ascii", "ignore").decode("utf-8")

    # Substitui todos os caracteres que não são letras por um espaço em branco
    s = re.sub(r'[^A-Za-z]', ' ', s)

    # Substitui espaços consecutivos por apenas um espaço
    s = re.sub(r'\s{2,}', ' ', s)

    # Remove os espaços em branco do começo e fim
    s = s.strip()

    return s

In [132]:
curriculos['cidade_nascimento_normalizada'] = curriculos.loc[:, 'cidade_nascimento'].apply(normalizar_string)

In [133]:
d = curriculos.loc[1490, ['siape', 'cidade_nascimento', 'cidade_nascimento_normalizada', 'uf_nascimento']]

for _, cidade in cidades.iterrows():
    if cidade['sigla_uf'] == d['uf_nascimento'] and (d['cidade_nascimento_normalizada'] == cidade['nome_ascii'] 
                                                     or d['cidade_nascimento'] in cidade['aliases']):
        d['codigo_cidade_nascimento'] = cidade['codigo']
        break

d

siape                                                         1288789
cidade_nascimento                                            Brasília
cidade_nascimento_normalizada                                brasilia
uf_nascimento                                                      DF
codigo_cidade_nascimento         a358418b-0e97-420c-85a7-0565b949f93b
Name: 1490, dtype: object

In [134]:
def link_curriculo_to_cidade(row, cidades):
    for _, cidade in cidades.iterrows():
        if row['cidade_nascimento_normalizada'] == cidade['nome_ascii'] or row['cidade_nascimento'] in cidade['aliases']:
            row['codigo_cidade_nascimento'] = cidade['codigo']
            break

    return row

In [135]:
curriculos = curriculos.apply(link_curriculo_to_cidade, args=(cidades,), axis=1)

In [136]:
curriculos = curriculos.drop(columns=['cidade_nascimento_normalizada'])

In [137]:
curriculos.sample(n=10)

,aceitando_email,campus_atual,categoria,cidade_nascimento,codigo,codigo_cidade_nascimento,data_nascimento,email,link,nome_citacoes_bibliograficas,nome_completo,nome_completo_sem_acento,palavra_chave,resumo,siape,sigla_campus_atual,uf_nascimento
1241,True,CÂMPUS FORMOSA,Docente,NATAL,651,a952b204-a839-45f1-81eb-f20ecddba97c,1988-03-25,andre.dantas@ifg.edu.br,http://lattes.cnpq.br/3443223409799104,"DANTAS, A. A. N.;DANTAS, ANDRÉ AUGUSTO NÓBREGA",Andre Augusto Nobrega Dantas,Andre Augusto Nobrega Dantas,"[{'palavra': 'estudo', 'correta': 'Estudo', 'i...",Possui graduação em Engenharia Civil pela Univ...,1230337,FOR,RN
959,True,CÂMPUS GOIÂNIA,Docente,Araçatuba,1447,24924ad9-8551-4acf-9ea6-ad072f2b68d0,1988-05-12,edipo.cremon@ifg.edu.br,http://lattes.cnpq.br/7888258901938956,"CREMON, É. H.;Cremon, Édipo Henrique;CREMON, E...",Édipo Henrique Cremon,Edipo Henrique Cremon,"[{'palavra': 'imagens', 'correta': 'Imagens', ...",Geógrafo pela Universidade Estadual de Maringá...,2285085,GYN,SP
888,True,CÂMPUS GOIÂNIA,Docente,Goiania,1338,8648f216-75f9-44b6-a00c-7d8f5a646c2d,1980-02-29,limerce.lopes@ifg.edu.br,http://lattes.cnpq.br/7266113954157102,"LOPES, L. F.",Limerce Ferreira Lopes,Limerce Ferreira Lopes,"[{'palavra': 'letramento', 'correta': 'Letrame...",Graduada em Letras com habilitação em Portuguê...,1796028,GYN,GO
609,True,REITORIA,TAE,Goiânia,895,8648f216-75f9-44b6-a00c-7d8f5a646c2d,1970-09-25,kenia.hidalgo@ifg.edu.br,http://lattes.cnpq.br/4708038363452351,"SILVA, K. R.;HIDALGO, K. R. da S.;HIDALGO, K. ...",Kenia Ribeiro da Silva Hidalgo,Kenia Ribeiro da Silva Hidalgo,"[{'palavra': 'aprendizagem', 'correta': 'Apren...",Doutora em Educação pela PUC-GO. Mestra em Edu...,3092355,REI,GO
305,True,CÂMPUS GOIÂNIA,TAE,GOIÂNIA,458,8648f216-75f9-44b6-a00c-7d8f5a646c2d,1985-01-23,camila.franco@ifg.edu.br,http://lattes.cnpq.br/2398750759537246,"FRANCO, C. N.",Camila Noleto Franco,Camila Noleto Franco,NaN,Possui graduação em Direito pela Pontifícia Un...,1649936,GYN,GO
1800,True,CÂMPUS ANÁPOLIS,TAE,Goiânia,840,8648f216-75f9-44b6-a00c-7d8f5a646c2d,1985-09-27,tales.tavares@ifg.edu.br,http://lattes.cnpq.br/4433615594721099,"TAVARES, T. O.",Tales de Oliveira Tavares,Tales de Oliveira Tavares,"[{'palavra': 'biofisica', 'correta': 'Biofísic...",Graduado no curso de Biologia com outorga de B...,1006480,ANA,GO
937,True,CÂMPUS CIDADE DE GOIÁS,Docente,Gurupi,1431,e224dd8f-9769-44b2-8509-f3c7c6f2a242,1981-11-11,alemar.sousa@ifg.edu.br,http://lattes.cnpq.br/7771133603438281,"SOUSA, Alemar Moreira de",Alemar Moreira de Sousa,Alemar Moreira de Sousa,NaN,possui graduação em Geografia pela Universidad...,2573433,GOI,TO
355,True,CÂMPUS ÁGUAS LINDAS,Docente,Rio Branco,533,9098d857-08f5-46ca-88d9-7c779ad9d65e,1981-07-19,rossemildo.santos@ifg.edu.br,http://lattes.cnpq.br/2886499316177159,"SANTOS, R. S.",Rossemildo da Silva Santos,Rossemildo da Silva Santos,NaN,Licenciatura em Letras Português-Espanhol e su...,1569507,LIN,AC
1708,True,CÂMPUS APARECIDA DE GOIÂNIA,TAE,Goiânia,513,8648f216-75f9-44b6-a00c-7d8f5a646c2d,1983-08-16,daniela.rezende@ifg.edu.br,http://lattes.cnpq.br/2725347320666971,"REZENDE, D. R.;REZENDE, DANIELA R.;REZENDE, D.R.",Daniela Rodrigues de Rezende,Daniela Rodrigues de Rezende,NaN,Daniela Rodrigues de Rezende é Técnica em Sane...,1941927,APA,GO
630,True,CÂMPUS JATAÍ,TAE,Jataí,925,a45d3350-91da-4db0-9883-104c996a85e1,1972-09-30,neiva.freitas@ifg.edu.br,http://lattes.cnpq.br/4914615367829274,"FREITAS, N. S.",Neiva Santos de Freitas,Neiva Santos de Freitas,NaN,Atualmente é Assistente social na Universidade...,1411262,JAT,GO


### Exportar

In [138]:
export_path = Path(IFG_PRODUZ_STORAGE_DIR / "preprocessed")

if not export_path.is_dir():
    Path(export_path).mkdir()

curriculos.to_csv(export_path / "curriculos.csv", sep=";", index=False)